In [3]:
import numpy as np
import tensorflow as tf
import pickle
import os

In [4]:
PROJECT_ROOT = "/home/joey/Work/thesis_code"
IMAGE_SIZE = 100
IMAGE_CHANNEL = 3
pickle_file = os.path.join(PROJECT_ROOT, "lung_cancer_data.pickle")

In [5]:
with open(pickle_file, "rb") as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save

In [6]:
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (7118, 100, 100, 3) (7118,)
Validation set (400, 100, 100, 3) (400,)
Test set (400, 100, 100, 3) (400,)


In [7]:
def reformat(data, labels):
    # don't fully understand the meaning of the parameter -1
    data = data.reshape((-1, IMAGE_CHANNEL * IMAGE_SIZE * IMAGE_SIZE)).astype(np.float32)
    labels = (np.arange(2) == labels[:, None]).astype(np.float32)
    return data, labels

In [8]:
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (7118, 30000) (7118, 2)
Validation set (400, 30000) (400, 2)
Test set (400, 30000) (400, 2)


Here in my model, w will have `30000*2=60000` elements, b will have 2 elements. The model will only give two results. Benign or Malignant

In [10]:
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.constant(train_dataset)
    tf_train_labels = tf.constant(train_labels)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    weights = tf.Variable(
        tf.truncated_normal([3 * IMAGE_SIZE * IMAGE_SIZE, 2]))
    biases = tf.Variable(tf.zeros(2))
    logits = tf.matmul(tf_train_dataset, weights) + biases
    # then turn logits into probabilities using softmax function
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    # loss: cross entropy between logits and label
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases)

In [ ]:
with tf.Session(graph=graph) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print("Initialized")
    